# **KoBERT 사용해보기**
https://hoit1302.tistory.com/159

https://github.com/SKTBrain/KoBERT

참고

# **1. Colab 환경을 설정하기**

In [ ]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=4938b9df9a9618fc9736eb173801c15cd4a6dfb534d47b82b761c94f2c98aec3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 235 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

# **2. github에서 KoBERT 파일을 로드 및 KoBERT모델 불러오기**

In [ ]:
# 토크나이저 설치
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-wsvv58dk/kobert-tokenizer_964bd2b868064c1392446591034358c4
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-wsvv58dk/kobert-tokenizer_964bd2b868064c1392446591034358c4
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=4de88ecb29ddb4a4cf2e3e99da617affd69ada9132decf603c31e516bf6bd961
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ahj8z07/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
# SKT KoBert 모델 설치
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zvuoyaa7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zvuoyaa7
     |████████████████████████████████| 132 kB 3.9 MB/s 
     |████████████████████████████████| 4.5 MB 8.0 MB/s 
     |████████████████████████████████| 8.9 MB 39.9 MB/s 
     |████████████████████████████████| 79 kB 4.8 MB/s 
     |████████████████████████████████| 127 kB 44.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=e23aaf1a0b528f437a675ef6639f950d7f7c89ad0b23b18abb534442747bf0e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-dkt_4ifq/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [ ]:
# 기본 kobert model 가져오기
from kobert.pytorch_kobert import get_pytorch_kobert_model
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
# 토크나이저 가져오기
from gluonnlp.data import SentencepieceTokenizer
from kobert import get_tokenizer
tok_path = get_tokenizer()
tok = SentencepieceTokenizer(tok_path)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# **3. 필요한 라이브러리 불러오기**

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# GPU 사용 시
device = torch.device("cuda:0")

# **4. 데이터셋 불러오기**

In [ ]:
#data1 = pd.read_csv('./4차년도.csv', encoding='cp949')
#data2 = pd.read_csv('./5차년도.csv', encoding='cp949')
#data3 = pd.read_csv('./5차년도_2차.csv', encoding='cp949')
data = pd.read_csv('./긍부정.csv', encoding='utf8')

#data = pd.concat([data1, data2])
#data = pd.concat([data, data3])
#data = pd.concat([data, data4])

data.head()

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5ed10dbc2880d70f286121c3,ㄱㅅ,positive,Neutral,0,Neutral,0,Happiness,1,Neutral,0,Fear,1,48,female
1,5ed10dbc2880d70f286121c3,ㄱㅇㄷ,positive,Neutral,0,Neutral,0,Happiness,1,Neutral,0,Fear,1,48,female
2,5ed10dbc2880d70f286121c3,가격이 싸다,positive,Neutral,0,Neutral,0,Happiness,1,Neutral,0,Fear,1,48,female
3,5ed10dbc2880d70f286121c3,가까스로,neutral,Neutral,0,Neutral,0,Happiness,1,Neutral,0,Fear,1,48,female
4,5ed10dbc2880d70f286121c3,가까이 사귀어,positive,Neutral,0,Neutral,0,Happiness,1,Neutral,0,Fear,1,48,female


In [ ]:
# 공포 : 0
data.loc[(data['상황'] == "fear"), '상황'] = 0
# 놀람 : 1
data.loc[(data['상황'] == "surprise"), '상황'] = 1
# 분노 : 2
data.loc[(data['상황'] == "anger"), '상황'] = 2
data.loc[(data['상황'] == "angry"), '상황'] = 2
# 슬픔 : 3
data.loc[(data['상황'] == "sad"), '상황'] = 3
data.loc[(data['상황'] == "sadness"), '상황'] = 3
# 중립 : 4
data.loc[(data['상황'] == "neutral"), '상황'] = 4 
# 행복 : 5
data.loc[(data['상황'] == "happiness"), '상황'] = 5
# 혐오 : 6
data.loc[(data['상황'] == "disgust"), '상황'] = 6
# 긍정 : 7
data.loc[(data['상황'] == "positive"), '상황'] = 7
# 부정 : 8
data.loc[(data['상황'] == "negative"), '상황'] = 8

In [ ]:
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)
    
#data_list

# **5. 입력 데이터셋을 토큰화하기**

In [ ]:
# Bert용 데이터 셋 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5        # 학습 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
# tok=tokenizer.tokenize
# 토크나이저 적용하여 Bert용 데이터 셋 형태로 변환
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# **6. KoBERT 모델 구현하기**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        
        ## Parent 클래스의 메서드 모두 가져오기 { Python 2.X Ver 문법 } -> { Python 3.x Ver 문법 : super().__init__() }
        ## 상속한 클래스 torch.nn.Module 
        super(BERTClassifier, self).__init__()
        
        self.bert = bert
        self.dr_rate = dr_rate         
        self.classifier = nn.Linear(hidden_size , num_classes)
        
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        
        attention_mask = torch.zeros_like(token_ids)
        
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        
        if self.dr_rate:
            out = self.dropout(pooler)
        
        return self.classifier(out)

In [ ]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

RuntimeError: ignored

# **7. Train**

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/550 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.229238510131836 train acc 0.125
epoch 1 batch id 201 loss 0.81678706407547 train acc 0.4400652985074627
epoch 1 batch id 401 loss 0.8610861897468567 train acc 0.6349750623441397
epoch 1 train acc 0.6988798701298701


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.930556773024778


  0%|          | 0/550 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.057894207537174225 train acc 0.96875
epoch 5 batch id 201 loss 0.25065866112709045 train acc 0.9725590796019901
epoch 5 batch id 401 loss 0.11584683507680893 train acc 0.9741661471321695
epoch 5 train acc 0.9751339285714284


  0%|          | 0/138 [00:00<?, ?it/s]

epoch 5 test acc 0.9296509759233287


# **8. 직접 만든 새로운 문장으로 테스트**

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 걱정해줘서 고마워


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 공포가 느껴집니다.


하고싶은 말을 입력해주세요 : 0


# **9. 모델 저장한 후 재사용**

In [ ]:
import joblib

joblib.dump(model, "bert_model.pkl")

['bert_model.pkl']

In [ ]:
import joblib

model = joblib.load("bert_model.pkl")

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 가즈아


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 가자
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 행복합니다
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 좋아요
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 좋습니다
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 싫어요
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 으....
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 저리가
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 아니
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 어쩌라구요
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 눈물이 난다
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 그래서요?
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 이거 되냐?
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 너 뭐 되냐?
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 흠 그런가?
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 그랬던가
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 대상에 대한 감정을 사실적으로 평가하는 것으로서 맥락에 따라 극성이 바뀔 수 있음.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 거짓말 절대 용서 못해!
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 대답해!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을